<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [ ]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [3]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# URL of the Falcon 9 Launch page
url = 'https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy'

# Perform the HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

# Optionally, you can print the HTML content of the page
# print(response.text)


Request was successful


Create a `BeautifulSoup` object from the HTML `response`


In [2]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

import requests
from bs4 import BeautifulSoup

# URL of the Falcon 9 Launch page
url = 'https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy'

# Perform the HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

# Create a BeautifulSoup object from the HTML response
soup = BeautifulSoup(response.text, 'html.parser')

# Optionally, you can print the prettified HTML content to inspect it
# print(soup.prettify())


Request was successful


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [3]:
# Use soup.title attribute

import requests
from bs4 import BeautifulSoup

# URL of the Falcon 9 Launch page
url = 'https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy'

# Perform the HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

# Create a BeautifulSoup object from the HTML response
soup = BeautifulSoup(response.text, 'html.parser')

# Print the page title to verify if the BeautifulSoup object was created properly
print(soup.title.text)


Request was successful
Anexo:Lanzamientos de cohetes Falcon 9 y Falcon Heavy - Wikipedia, la enciclopedia libre


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [4]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`

# Re-running the required code to extract tables from the HTML page using BeautifulSoup
from bs4 import BeautifulSoup
import requests

# Request the Falcon 9 Launch Wikipedia page
url = "https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy"
response = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables in the BeautifulSoup object
tables = soup.find_all('table')

# Display the number of tables found
len(tables)



14

Starting from the third table is our target table contains the actual launch records.


In [6]:
# Let's print the third table and check its content
import requests
from bs4 import BeautifulSoup

# Make an HTTP GET request to the Falcon 9 launch page
url = "https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy"
response = requests.get(url)

# Create the BeautifulSoup object from the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table', class_='wikitable')

# Target the third table (index 2 in Python, since index starts at 0)
target_table = tables[2]

# Get the column names from the table header (th)
headers = [th.text.strip() for th in target_table.find_all('th')]

# Get all rows (excluding the header) and extract the data for each cell
rows = target_table.find_all('tr')[1:]  # Skip the first row (header row)

# Extract the data for each row
data = []
for row in rows:
    cells = row.find_all('td')
    cell_data = [cell.text.strip() for cell in cells]
    data.append(cell_data)

# Display the headers and the first few rows of data
print("Headers:", headers)
print("First 5 rows of data:")
for row in data[:5]:
    print(row)


Headers: ['Vuelo #', 'Fecha y hora (UTC)', 'Versión, propulsor[b]\u200b', 'Sitio de lanzamiento', 'Carga útil', 'Masa de carga útil', 'Órbita', 'Cliente', 'Resultado del lanzamiento', 'Aterrizaje del propulsor', '14', '15', '16', '17', '18', '19', '20']
First 5 rows of data:
['10 de enero de 2015 09:47[70]\u200b', 'F9 v1.1 B1012[12]\u200b', 'CCAFSSLC-40', 'SpaceX CRS-5[71]\u200b', '2.395 kg[72]\u200b[c]\u200b', 'OTB (ISS)', 'NASA (CRS)', 'Éxito[73]\u200b', 'Fallo[74]\u200b  (JRTI)']
['Luego de la separación de la segunda etapa, SpaceX intentó devolver la primera etapa a una plataforma flotante 90 por 50 metros, llamada «plataforma autónoma de puerto aeroespacial» (Autonomous Spaceport Drone Ship, ASDS.) La prueba logró muchos objetivos y devolvió una gran cantidad de datos, pero las superficies de control de aletas de rejilla utilizadas por primera vez para un posicionamiento de reentrada más preciso se quedaron sin fluido hidráulico para su sistema de control un minuto antes del aterr

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [7]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

import requests
from bs4 import BeautifulSoup

# Function to clean and extract column names from the header
def extract_column_from_header(header):
    # Clean and format the column name (strip extra spaces and handle any special characters if needed)
    return header.text.strip()

# Make an HTTP GET request to the Falcon 9 launch page
url = "https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy"
response = requests.get(url)

# Create the BeautifulSoup object from the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table', class_='wikitable')

# Target the third table (index 2 in Python, since index starts at 0)
target_table = tables[2]

# Get the column names from the table header (th) and apply the extract_column_from_header function
headers = [extract_column_from_header(th) for th in target_table.find_all('th')]

# Display the extracted headers
print("Extracted Headers:", headers)



Extracted Headers: ['Vuelo #', 'Fecha y hora (UTC)', 'Versión, propulsor[b]\u200b', 'Sitio de lanzamiento', 'Carga útil', 'Masa de carga útil', 'Órbita', 'Cliente', 'Resultado del lanzamiento', 'Aterrizaje del propulsor', '14', '15', '16', '17', '18', '19', '20']


Check the extracted column names


In [9]:
print(column_names)
import requests
from bs4 import BeautifulSoup

# Function to clean and extract column names from the header
def extract_column_from_header(header):
    return header.text.strip()

# Make an HTTP GET request to the Falcon 9 launch page
url = "https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy"
response = requests.get(url)

# Create the BeautifulSoup object from the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table', class_='wikitable')

# Target the third table (index 2 in Python, since index starts at 0)
target_table = tables[2]

# Get the column names from the table header (th) and apply the extract_column_from_header function
headers = [extract_column_from_header(th) for th in target_table.find_all('th')]

# Display the extracted headers
print("Extracted Headers:", headers)


[]
Extracted Headers: ['Vuelo #', 'Fecha y hora (UTC)', 'Versión, propulsor[b]\u200b', 'Sitio de lanzamiento', 'Carga útil', 'Masa de carga útil', 'Órbita', 'Cliente', 'Resultado del lanzamiento', 'Aterrizaje del propulsor', '14', '15', '16', '17', '18', '19', '20']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [12]:
import pandas as pd

# Extracted column names from the previous task (this is just a placeholder, use your actual list of column names)
column_names = ['Flight Number', 'Launch Date', 'Rocket Type', 'Payload', 'Orbit', 'Landing', 'Outcome']  # Example names

# Create an empty dictionary with keys as column names
data_dict = {column: [] for column in column_names}

# Convert the dictionary into a Pandas DataFrame
df = pd.DataFrame(data_dict)

# Display the empty DataFrame
print(df)



Empty DataFrame
Columns: [Flight Number, Launch Date, Rocket Type, Payload, Orbit, Landing, Outcome]
Index: []


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [14]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            
            
            import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Wikipedia page with Falcon 9 and Falcon Heavy launches
url = "https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy"

# Perform HTTP GET request to get the page content
response = requests.get(url)

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the tables on the page
tables = soup.find_all('table', {'class': 'wikitable'})

# Select the third table (index 2), which contains the actual launch data
target_table = tables[2]

# Initialize the launch_dict with the expected column names (customize as needed)
launch_dict = {
    'Flight Number': [],
    'Launch Date': [],
    'Rocket Type': [],
    'Payload': [],
    'Orbit': [],
    'Landing': [],
    'Outcome': []
}

# Iterate over each row in the table, starting from the second row (skip header)
for row in target_table.find_all('tr')[1:]:
    cols = row.find_all('td')  # Find all <td> elements in the row
    
    # Ensure there are enough columns in this row (e.g., 7 columns)
    if len(cols) >= 7:
        # Extract each piece of data and clean up the text by stripping unwanted spaces
        flight_number = cols[0].get_text(strip=True)
        launch_date = cols[1].get_text(strip=True)
        rocket_type = cols[2].get_text(strip=True)
        payload = cols[3].get_text(strip=True)
        orbit = cols[4].get_text(strip=True)
        landing = cols[5].get_text(strip=True)
        outcome = cols[6].get_text(strip=True)
        
        # Append the extracted values to the corresponding lists in the launch_dict
        launch_dict['Flight Number'].append(flight_number)
        launch_dict['Launch Date'].append(launch_date)
        launch_dict['Rocket Type'].append(rocket_type)
        launch_dict['Payload'].append(payload)
        launch_dict['Orbit'].append(orbit)
        launch_dict['Landing'].append(landing)
        launch_dict['Outcome'].append(outcome)

# Convert the launch_dict into a Pandas DataFrame
df = pd.DataFrame(launch_dict)

# Display the DataFrame to verify the data
print(df)

            

                       Flight Number        Launch Date  Rocket Type  \
0      10 de enero de 201509:47[70]​  F9 v1.1B1012[12]​  CCAFSSLC-40   
1    11 de febrero de 201523:03[77]​       F9 v1.1B1013  CCAFSSLC-40   
2  2 de marzo de 201503:50[26]​[80]​       F9 v1.1B1014  CCAFSSLC-40   
3           14 de abril de 201520:10       F9 v1.1B1015  CCAFSSLC-40   
4      27 de abril de 201523:03[88]​       F9 v1.1B1016  CCAFSSLC-40   
5           28 de junio de 201514:21       F9 v1.1B1018  CCAFSSLC-40   
6  22 de diciembre de 201501:29[97]​         F9 FTB1019  CCAFSSLC-40   

                             Payload              Orbit  \
0                  SpaceX CRS-5[71]​  2.395 kg[72]​[c]​   
1                        DSCOVR[78]​             570 kg   
2     ABS-3AEutelsat 115 West B[71]​           4.159 kg   
3                       SpaceX CRS-6  1.898 kg[84]​[c]​   
4  TürkmenÄlem 52°E / MonacoSAT[89]​           4.707 kg   
5                       SpaceX CRS-7  1.952 kg[92]​[c]​   
6   Orbcom

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [16]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Wikipedia page with Falcon 9 and Falcon Heavy launches
url = "https://es.wikipedia.org/wiki/Anexo:Lanzamientos_de_cohetes_Falcon_9_y_Falcon_Heavy"

# Perform HTTP GET request to get the page content
response = requests.get(url)

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the tables on the page
tables = soup.find_all('table', {'class': 'wikitable'})

# Select the third table (index 2), which contains the actual launch data
target_table = tables[2]

# Initialize the launch_dict with the expected column names
launch_dict = {
    'Flight Number': [],
    'Launch Date': [],
    'Rocket Type': [],
    'Payload': [],
    'Orbit': [],
    'Landing': [],
    'Outcome': []
}

# Iterate over each row in the table, starting from the second row (skip header)
for row in target_table.find_all('tr')[1:]:
    cols = row.find_all('td')  # Find all <td> elements in the row
    
    # Ensure there are enough columns in this row (e.g., 7 columns)
    if len(cols) >= 7:
        # Extract each piece of data and clean up the text by stripping unwanted spaces
        flight_number = cols[0].get_text(strip=True)
        launch_date = cols[1].get_text(strip=True)
        rocket_type = cols[2].get_text(strip=True)
        payload = cols[3].get_text(strip=True)
        orbit = cols[4].get_text(strip=True)
        landing = cols[5].get_text(strip=True)
        outcome = cols[6].get_text(strip=True)
        
        # Append the extracted values to the corresponding lists in the launch_dict
        launch_dict['Flight Number'].append(flight_number)
        launch_dict['Launch Date'].append(launch_date)
        launch_dict['Rocket Type'].append(rocket_type)
        launch_dict['Payload'].append(payload)
        launch_dict['Orbit'].append(orbit)
        launch_dict['Landing'].append(landing)
        launch_dict['Outcome'].append(outcome)

# Convert the launch_dict into a Pandas DataFrame
df = pd.DataFrame(launch_dict)

# Display the DataFrame to verify the data
print(df)


                       Flight Number        Launch Date  Rocket Type  \
0      10 de enero de 201509:47[70]​  F9 v1.1B1012[12]​  CCAFSSLC-40   
1    11 de febrero de 201523:03[77]​       F9 v1.1B1013  CCAFSSLC-40   
2  2 de marzo de 201503:50[26]​[80]​       F9 v1.1B1014  CCAFSSLC-40   
3           14 de abril de 201520:10       F9 v1.1B1015  CCAFSSLC-40   
4      27 de abril de 201523:03[88]​       F9 v1.1B1016  CCAFSSLC-40   
5           28 de junio de 201514:21       F9 v1.1B1018  CCAFSSLC-40   
6  22 de diciembre de 201501:29[97]​         F9 FTB1019  CCAFSSLC-40   

                             Payload              Orbit  \
0                  SpaceX CRS-5[71]​  2.395 kg[72]​[c]​   
1                        DSCOVR[78]​             570 kg   
2     ABS-3AEutelsat 115 West B[71]​           4.159 kg   
3                       SpaceX CRS-6  1.898 kg[84]​[c]​   
4  TürkmenÄlem 52°E / MonacoSAT[89]​           4.707 kg   
5                       SpaceX CRS-7  1.952 kg[92]​[c]​   
6   Orbcom

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [17]:
# Export the DataFrame to a CSV file for the next section
df.to_csv('falcon9_launches.csv', index=False)

# Confirmation message
print("Data successfully exported to 'falcon9_launches.csv'. You can use this file for the next section.")


Data successfully exported to 'falcon9_launches.csv'. You can use this file for the next section.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
